### a) Formulación del modelo de transporte

1. **Variables de decisión:**
   * $x_{ij}$ será el número de automóviles que se envían desde el centro $i$ al concesionario $j$.
   * $i = 1,2,3$
   * $j = 1,2,3,4,5$

2. **Función objetivo**
    La función objetivo será el costo total, el cual es:
    $$\sum_{i=1}^{3} \sum_{j=1}^{5} x_{ij} * c_{ij}$$

    El objetivo es minimizar la función de costo, debido a que el costo por unidad transportada será menor del centro $i$ al concesionario $j$. Esta se obtiene multiplicando la distancia por el costo por unidad de distancia (millas) entre la cantidad de autos por carga $(18)$.

3. **Restricciones:**
   
   ***Oferta***
   
    Obteniendo los componentes de la suma de costos tenemos que la cantidad total enviada desde un centro es
    $$\sum_{j=1}^{5}x_{ij}$$

    Entonces la cantidad total enviada no debe exceder la oferta
    $$\sum_{j=1}^{5}x_{ij} \leq sale_{i} \ \ \ \ para \ i = 1,2,3$$

    ***Demanda***

    Obteniendo los componentes de la suma de costos tenemos que la cantidad total recibida por un concesionario es:
    $$\sum_{i=1}^{3}x_{ij}$$

    Entonces la cantidad total recibida por un concesionario debe ser igual a la demanda
    $$\sum_{i=1}^{3}x_{ij} = demanda_{j} \ \ \ \ para \ j = 1, 2, 3, 4, 5$$



4. **No negatividad:**
   Las cantidades en las sumas no pueden ser negativas
   $$x_{ij} \geq 0 \ \ \ \ \ \forall \ \ i, j$$

### b) Código

In [1]:
using JuMP, GLPK

**Datos**

In [2]:
costPMile = 25
maxCars = 18

18

**Distnacias entre concesionarios y centros**

In [3]:
dists = [
    100 150 200 140 35;
    50 70 60 65 80;
    40 90 100 150 130;
]

3×5 Matrix{Int64}:
 100  150  200  140   35
  50   70   60   65   80
  40   90  100  150  130

**Ofertas y demandas**

In [4]:
offers = [400, 200, 150]
demands = [100, 200, 150, 160, 200]

5-element Vector{Int64}:
 100
 200
 150
 160
 200

**Creación del modelo**

In [5]:
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

**Conjunto de variables**

In [6]:
@variable(model, x[1:3, 1:5] >= 0)

3×5 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]

**Función a minimizar (objetivo)**

In [7]:
@objective(model, Min, sum(x[i,j] * dists[i,j] * costPMile / maxCars for i in 1:3, j in 1:5))

138.88888888888889 x[1,1] + 208.33333333333334 x[1,2] + 277.77777777777777 x[1,3] + 194.44444444444446 x[1,4] + 48.611111111111114 x[1,5] + 69.44444444444444 x[2,1] + 97.22222222222223 x[2,2] + 83.33333333333333 x[2,3] + 90.27777777777777 x[2,4] + 111.11111111111111 x[2,5] + 55.55555555555556 x[3,1] + 125 x[3,2] + 138.88888888888889 x[3,3] + 208.33333333333334 x[3,4] + 180.55555555555554 x[3,5]

**Restricciones**

In [8]:
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:5) <= offers[i])
end

for j in 1:5
    @constraint(model, sum(x[i,j] for i in 1:3) == demands[j])
end

**Resolución del modelo**

In [9]:
optimize!(model)

**Presentar resultados**

In [10]:
if termination_status(model) == MOI.OPTIMAL
    println("Distribución óptima:")
    for i in 1:3, j in 1:5
        println("Enviar ", value(x[i,j]), " autos desde el Centro ", i, " al Concesionario ", j)
    end
    println("Costo total: \$", objective_value(model))
else
    println("No se encontró solución óptima.")
end

No se encontró solución óptima.
